In [1]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    env = gym.wrappers.Monitor(env, directory="Recording", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print(f"{iter_no}: loss={loss_v.item()}, reward_mean={reward_m}, reward_bound={reward_b}")
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()

C:\Users\thisi\anaconda3\lib\site-packages\pyglet\window\__init__.py:667: UserWarning: 
Your graphics drivers do not support OpenGL 2.0.
You may experience rendering issues or crashes.
Microsoft Corporation
GDI Generic
1.1.0
  warnings.warn(message)
C:\Users\thisi\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))
<ipython-input-1-e77ec10e644e>:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  obs_v = torch.FloatTensor([obs])


0: loss=0.69057697057724, reward_mean=22.9375, reward_bound=26.5
1: loss=0.6748207211494446, reward_mean=24.875, reward_bound=29.0
2: loss=0.660508394241333, reward_mean=27.625, reward_bound=29.0
3: loss=0.6534098982810974, reward_mean=28.125, reward_bound=31.5
4: loss=0.634917676448822, reward_mean=43.625, reward_bound=53.5
5: loss=0.620573103427887, reward_mean=56.6875, reward_bound=67.5
6: loss=0.6426928043365479, reward_mean=45.125, reward_bound=47.0
7: loss=0.6131179332733154, reward_mean=58.125, reward_bound=70.0
8: loss=0.5998068451881409, reward_mean=40.625, reward_bound=45.5
9: loss=0.6000617742538452, reward_mean=54.6875, reward_bound=69.0
10: loss=0.5917733311653137, reward_mean=64.875, reward_bound=73.5
11: loss=0.6076008081436157, reward_mean=63.1875, reward_bound=71.5
12: loss=0.5849156379699707, reward_mean=59.6875, reward_bound=70.5
13: loss=0.5735911130905151, reward_mean=67.1875, reward_bound=79.5
14: loss=0.5772327780723572, reward_mean=71.5, reward_bound=69.0
15: lo